# Линейная регрессия 
(*реализация через градиентный спуск без сторонних библиотек*)

Пусть у нас есть некоторая функция потерь $f:\mathbb{X}\times\mathbb{Y}\rightarrow\mathbb{R}$ и матрица признаков $X$ c целевой переменной $y$, тогда функционал ошибки выглядит так: $$L(f,X,y)=\frac{1}{N}\sum_{i=1}^{N}{f(\vec{x}_i,y_i)}$$
В нашем случае $f(\vec{x},y)=(\langle{\vec{x},w}\rangle-y)^2$, а значит: $$L(f,X,y)=\frac{1}{N}\sum_{i=1}^{N}{(\langle{\vec{x}_i,w}\rangle-y_i)^2}$$
Реализуем:

In [36]:
from typing import List
import csv,math,time

def loss_function(w:List[float],x:List[float],y:float)->float:
    return (sum(list(map(lambda a,b:a*b,w,x)))-y)**2

def error_functionality(w:List[float],X:List[List[float]],y:List[float])->float:
    sum=0
    for i in range(len(X)):
        sum+=loss_function(w,X[i],y[i])
    return sum/len(X)

Загружаем данные:

In [37]:
data,X,y = [],[],[]
with open('my_data.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Преобразуем каждый элемент строки в float
        data.append([float(value) for value in row])
        X.append(data[-1][:-1])
        y.append(data[-1][-1])
w=len(X[0])*[1]
print('Первые 5 обьектов: ',data[:5])

Первые 5 обьектов:  [[254.5200092, 102.1921871], [351.2964474, 89.79305002], [36.83525135, 330.3657023], [346.4289733, 113.03682], [475.2232841, 320.1528027]]


Функции, которые немного изменены для корректной работы с данными из прошлого дневника:

In [38]:
#Реализация градиента
def grad(f, w:List[float],X:List[List[float]],y:List[float], eps=10**(-5))->List[float]:
    w_dev=[]
    for i in range(len(w)):
        w_1,w_2=w.copy(),w.copy()
        w_1[i]-=eps
        w_2[i]+=eps
        w_dev.append((f(w_2,X,y)-f(w_1,X,y))/(2*eps))
    return w_dev
    
#Реализация градиентного спуска
def grad_desc(f,w:List[float],X:List[List[float]],y:List[float],learning_rate=0.01,iterations=100000)->List[float]:
    for i in range(iterations):
        w=list(map(lambda a,l,g:a-l*g,w,len(w)*[learning_rate],grad(f,w,X,y)))
    return [round(i,5) for i in w]

Минимизируем функционал ошибки с помощью градиентного спуска:

In [39]:
start=time.time()
print('Коэффициенты w:',grad_desc(error_functionality,w,X,y,0.00001,1000))
end=time.time()
print('Потраченное время:', round(end-start,3),'сек')

Коэффициенты w: [0.78374]
Потраченное время: 1.357 сек


Проверим коэффициенты библиотекой *sklearn*.

In [40]:
from sklearn.linear_model import LinearRegression
model=LinearRegression(fit_intercept=False)
start=time.time()
model.fit(X,y)
print('Коэффициенты w:',model.coef_)
end=time.time()
print('Потраченное время:', round(end-start,3),'сек')

Коэффициенты w: [0.78374364]
Потраченное время: 0.001 сек


Заметим, что моя реализация очень медленная по сравнению с Линейной регрессией из *sklearn*, т.к. она написана на языке C, который гораздо быстрее Python. Всё равно предлагаю более быстрый подход.
#### Матричное дифференцирование
$$L(w)=\frac{1}{N}\sum_{i=1}^{N}{(\langle{\vec{x}_i,w}\rangle-y_i)^2}$$
$$\nabla_w{L(w)}=\frac{2}{N}\sum_{i=1}^{N}{(\langle{\vec{x}_i,w}\rangle-y_i)}\vec{x}_i$$

In [41]:
def analytical_grad(w:List[float],X:List[List[float]],y:List[float])->List[float]:
    gradient=[0]*len(w)
    for i in range(len(X)):
        C=2*(sum(list(map(lambda a,b:a*b,w,X[i])))-y[i])/len(X)
        gradient=list(map(lambda g,k,v:g+k*v,gradient,len(w)*[C],X[i]))
    return gradient

def analytical_grad_desc(w:List[float],X:List[List[float]],y:List[float],learning_rate=0.01,iterations=100000)->List[float]:
    for i in range(iterations):
        w=list(map(lambda a,l,g:a-l*g,w,len(w)*[learning_rate],analytical_grad(w,X,y)))
    return [round(i,5) for i in w]

w=len(X[0])*[1]
start=time.time()
print('Коэффициенты w:',analytical_grad_desc(w,X,y,0.00001,1000))
end=time.time()
print('Потраченное время:', round(end-start,3),'сек')

Коэффициенты w: [0.78374]
Потраченное время: 1.171 сек
